<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Cross_Tab_with_OLAP_Cube_SQL_Superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

Reference : Please see [this slide deck](https://docs.google.com/presentation/d/1wQzZoHFo3NVk-dMTebHA4LH7B4KYmOfiSammjsSRGL0/edit?usp=sharing)

#Install MySQL

In [1]:
!apt update > /dev/null
!apt install -y  mysql-server > /dev/null

In [2]:
!/etc/init.d/mysql restart

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.


In [3]:
!mysql -e 'create database praxisDB'
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


#Create and Load Table

In [4]:
!mysql praxisDB -e 'drop table if exists ss_order'
# --------------------------------------------------------------
!mysql praxisDB -e '\
CREATE TABLE IF NOT EXISTS ss_order (\
RowID int(4),\
OrderID char(14),OrderDate date,\
ShipDate date,ShipMode varchar(16),\
CustomerID char(8),CustomerName varchar(30),Segment varchar(20),\
Country varchar(30),City varchar(30),State varchar(30),PostalCode char(5),Region varchar(15) ,\
ProductID varchar(20), Category varchar(40), SubCategory varchar(40), ProductName varchar(200), \
Sales decimal(8,2), Quantity int(4), Discount decimal(4,2), Profit decimal(8,2) \
); \
'
# --------------------------------------------------------------
!mysql praxisDB -e 'show tables'


+--------------------+
| Tables_in_praxisDB |
+--------------------+
| ss_order           |
+--------------------+


In [5]:
!mysql praxisDB -e 'desc ss_order'

+--------------+--------------+------+-----+---------+-------+
| Field        | Type         | Null | Key | Default | Extra |
+--------------+--------------+------+-----+---------+-------+
| RowID        | int(4)       | YES  |     | NULL    |       |
| OrderID      | char(14)     | YES  |     | NULL    |       |
| OrderDate    | date         | YES  |     | NULL    |       |
| ShipDate     | date         | YES  |     | NULL    |       |
| ShipMode     | varchar(16)  | YES  |     | NULL    |       |
| CustomerID   | char(8)      | YES  |     | NULL    |       |
| CustomerName | varchar(30)  | YES  |     | NULL    |       |
| Segment      | varchar(20)  | YES  |     | NULL    |       |
| Country      | varchar(30)  | YES  |     | NULL    |       |
| City         | varchar(30)  | YES  |     | NULL    |       |
| State        | varchar(30)  | YES  |     | NULL    |       |
| PostalCode   | char(5)      | YES  |     | NULL    |       |
| Region       | varchar(15)  | YES  |     | NULL    | 

Super Store Order Data https://drive.google.com/file/d/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ/view?usp=sharing <br>
Original Data from [Tableau](https://community.tableau.com/docs/DOC-1236)

In [6]:
#!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'
#!gdown https://drive.google.com/uc?id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
!wget -O SS_Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'

--2021-10-29 02:28:31--  https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2302058 (2.2M) [text/plain]
Saving to: ‘SS_Orders.csv’

SS_Orders.csv       100%[===================>]   2.20M  --.-KB/s    in 0.08s   

2021-10-29 02:28:31 (29.2 MB/s) - ‘SS_Orders.csv’ saved [2302058/2302058]



In [ ]:
#!mysql praxisDB -e "   \
#LOAD DATA LOCAL INFILE 'SS_Orders_1.csv' INTO TABLE ss_order \
#FIELDS TERMINATED BY ',' LINES TERMINATED BY '\r\n' IGNORE 1 LINES;  \
#"

In [34]:
!mysql praxisDB -e "   \
LOAD DATA LOCAL INFILE 'SS_Orders.csv' INTO TABLE ss_order \
FIELDS TERMINATED BY ',' enclosed by '\"' IGNORE 1 LINES;  \
"

In [35]:
!mysql praxisDB -e 'select count(*) from ss_order;'

+----------+
| count(*) |
+----------+
|     9994 |
+----------+


In [36]:
!mysql praxisDB -e 'select * from ss_order limit 10'
#the date data would not have loaded correctly 

+-------+----------------+------------+------------+----------------+------------+-----------------+-----------+---------------+-----------------+------------+------------+--------+-----------------+-----------------+-------------+------------------------------------------------------------------+--------+----------+----------+---------+
| RowID | OrderID        | OrderDate  | ShipDate   | ShipMode       | CustomerID | CustomerName    | Segment   | Country       | City            | State      | PostalCode | Region | ProductID       | Category        | SubCategory | ProductName                                                      | Sales  | Quantity | Discount | Profit  |
+-------+----------------+------------+------------+----------------+------------+-----------------+-----------+---------------+-----------------+------------+------------+--------+-----------------+-----------------+-------------+------------------------------------------------------------------+--------+----------+--

Let us fix the date loading error

In [37]:
!head SS_Orders.csv
# note that date is dd/mm/yyyy format

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United Stat

In [38]:
!mysql praxisDB -e "truncate ss_order;"
!mysql praxisDB -e 'select count(*) from ss_order;'

+----------+
| count(*) |
+----------+
|        0 |
+----------+


In [39]:
# see https://stackoverflow.com/questions/18927249/how-to-load-date-data-in-mysql-when-using-load-data
#
!mysql praxisDB -e "   \
LOAD \
DATA LOCAL INFILE 'SS_Orders.csv' \
INTO TABLE ss_order \
FIELDS TERMINATED BY ',' \
enclosed by '\"' \
IGNORE 1 LINES  \
(RowID, OrderID, @OrderDate, @ShipDate, ShipMode, CustomerID, CustomerName, Segment, \
 Country, City, State, PostalCode, Region, ProductID, Category, SubCategory, \
 ProductName, Sales, Quantity, Discount, Profit) \
set \
OrderDate = str_to_date(@OrderDate, '%d/%m/%Y'), \
ShipDate = str_to_date(@ShipDate, '%d/%m/%Y'); \
"

In [40]:
!mysql praxisDB -e 'select * from ss_order limit 10'
#the date data should have loaded correctly

+-------+----------------+------------+------------+----------------+------------+-----------------+-----------+---------------+-----------------+------------+------------+--------+-----------------+-----------------+-------------+------------------------------------------------------------------+--------+----------+----------+---------+
| RowID | OrderID        | OrderDate  | ShipDate   | ShipMode       | CustomerID | CustomerName    | Segment   | Country       | City            | State      | PostalCode | Region | ProductID       | Category        | SubCategory | ProductName                                                      | Sales  | Quantity | Discount | Profit  |
+-------+----------------+------------+------------+----------------+------------+-----------------+-----------+---------------+-----------------+------------+------------+--------+-----------------+-----------------+-------------+------------------------------------------------------------------+--------+----------+--

# Basic Query

In [ ]:
!mysql praxisDB -e 'select count(Quantity) from ss_order;'

+-----------------+
| count(Quantity) |
+-----------------+
|            9994 |
+-----------------+


In [ ]:
#!mysql praxisDB -e 'select count(*),sum(Quantity),Sum(Sales), Sum(Profit) from ss_order;'

In [ ]:
!mysql praxisDB -e 'select Category, Sum(Sales), Sum(Profit) from ss_order group by Category;'

+-----------------+------------+-------------+
| Category        | Sum(Sales) | Sum(Profit) |
+-----------------+------------+-------------+
| Furniture       |  741999.98 |    18451.25 |
| Office Supplies |  719046.99 |   122490.78 |
| Technology      |  836154.10 |   145455.64 |
+-----------------+------------+-------------+


In [ ]:
!mysql praxisDB -e 'select Region, Sum(Sales), Sum(Profit) from ss_order group by Region;'

+---------+------------+-------------+
| Region  | Sum(Sales) | Sum(Profit) |
+---------+------------+-------------+
| Central |  501239.88 |    39706.44 |
| East    |  678781.36 |    91522.80 |
| South   |  391721.90 |    46749.69 |
| West    |  725457.93 |   108418.74 |
+---------+------------+-------------+


In [ ]:
!mysql praxisDB -e 'select Segment, Sum(Sales), Sum(Profit) from ss_order group by Segment;'

+-------------+------------+-------------+
| Segment     | Sum(Sales) | Sum(Profit) |
+-------------+------------+-------------+
| Consumer    | 1161401.34 |   134119.27 |
| Corporate   |  706146.44 |    91979.41 |
| Home Office |  429653.29 |    60298.99 |
+-------------+------------+-------------+


#Complex Query

In [ ]:
!mysql praxisDB -e 'select Segment, Region, Sum(Sales) Sales from ss_order group by Segment, Region;'

+-------------+---------+-----------+
| Segment     | Region  | Sales     |
+-------------+---------+-----------+
| Consumer    | Central | 252031.43 |
| Consumer    | East    | 350908.17 |
| Consumer    | South   | 195580.92 |
| Consumer    | West    | 362880.82 |
| Corporate   | Central | 157995.79 |
| Corporate   | East    | 200409.45 |
| Corporate   | South   | 121885.95 |
| Corporate   | West    | 225855.25 |
| Home Office | Central |  91212.66 |
| Home Office | East    | 127463.74 |
| Home Office | South   |  74255.03 |
| Home Office | West    | 136721.86 |
+-------------+---------+-----------+


In [ ]:
!mysql praxisDB -e 'select Segment, Region, Sum(Sales) Sales from ss_order group by Segment, Region with Rollup;'

+-------------+---------+------------+
| Segment     | Region  | Sales      |
+-------------+---------+------------+
| Consumer    | Central |  252031.43 |
| Consumer    | East    |  350908.17 |
| Consumer    | South   |  195580.92 |
| Consumer    | West    |  362880.82 |
| Consumer    | NULL    | 1161401.34 |
| Corporate   | Central |  157995.79 |
| Corporate   | East    |  200409.45 |
| Corporate   | South   |  121885.95 |
| Corporate   | West    |  225855.25 |
| Corporate   | NULL    |  706146.44 |
| Home Office | Central |   91212.66 |
| Home Office | East    |  127463.74 |
| Home Office | South   |   74255.03 |
| Home Office | West    |  136721.86 |
| Home Office | NULL    |  429653.29 |
| NULL        | NULL    | 2297201.07 |
+-------------+---------+------------+


#Cross Tab

In [ ]:
!mysql praxisDB -e "\
SELECT \
  IFNULL(Segment,'Totals') 'Segment, Region', \
  sums.Central, sums.East, sums.West,sums.South,\
  sums.Central + sums.East + sums.West + sums.South Totals \
FROM ( \
  SELECT \
	Segment,\
	SUM(IF(Region='Central',Sales,0)) As 'Central',\
	SUM(IF(Region='East',Sales,0)) As 'East',\
    SUM(IF(Region='West',Sales,0)) As 'West',\
	SUM(IF(Region='South',Sales,0)) As 'South'\
  FROM ss_order \
  GROUP BY Segment WITH ROLLUP \
) AS sums"

+-----------------+-----------+-----------+-----------+-----------+------------+
| Segment, Region | Central   | East      | West      | South     | Totals     |
+-----------------+-----------+-----------+-----------+-----------+------------+
| Consumer        | 252031.43 | 350908.17 | 362880.82 | 195580.92 | 1161401.34 |
| Corporate       | 157995.79 | 200409.45 | 225855.25 | 121885.95 |  706146.44 |
| Home Office     |  91212.66 | 127463.74 | 136721.86 |  74255.03 |  429653.29 |
| Totals          | 501239.88 | 678781.36 | 725457.93 | 391721.90 | 2297201.07 |
+-----------------+-----------+-----------+-----------+-----------+------------+


In [ ]:
!mysql praxisDB -e "\
SELECT \
  IFNULL(Segment,'Totals') AS 'Segment, Category', \
  sums.Furniture, sums.Office_Supplies, sums.Technology,\
  sums.Furniture + sums.Office_Supplies + sums.Technology AS Totals \
  FROM (\
  SELECT\
	Segment,\
	SUM(IF(Category='Furniture',Sales,0)) As 'Furniture',\
	SUM(IF(Category='Office Supplies',Sales,0)) As 'Office_Supplies',\
	SUM(IF(Category='Technology',Sales,0)) As 'Technology'\
  FROM ss_order \
  GROUP BY Segment WITH ROLLUP\
) AS sums \
"


+-------------------+-----------+-----------------+------------+------------+
| Segment, Category | Furniture | Office_Supplies | Technology | Totals     |
+-------------------+-----------+-----------------+------------+------------+
| Consumer          | 391049.43 |       363951.96 |  406399.95 | 1161401.34 |
| Corporate         | 229019.83 |       230676.49 |  246450.12 |  706146.44 |
| Home Office       | 121930.72 |       124418.54 |  183304.03 |  429653.29 |
| Totals            | 741999.98 |       719046.99 |  836154.10 | 2297201.07 |
+-------------------+-----------+-----------------+------------+------------+


In [ ]:
!mysql praxisDB -e "\
SELECT\
	Segment,\
	SUM(IF(Category='Furniture',Sales,0)) As 'Furniture',\
	SUM(IF(Category='Office Supplies',Sales,0)) As 'Office_Supplies',\
	SUM(IF(Category='Technology',Sales,0)) As 'Technology'\
  FROM ss_order \
  GROUP BY Segment WITH ROLLUP\
  "

+-------------+-----------+-----------------+------------+
| Segment     | Furniture | Office_Supplies | Technology |
+-------------+-----------+-----------------+------------+
| Consumer    | 391049.43 |       363951.96 |  406399.95 |
| Corporate   | 229019.83 |       230676.49 |  246450.12 |
| Home Office | 121930.72 |       124418.54 |  183304.03 |
| NULL        | 741999.98 |       719046.99 |  836154.10 |
+-------------+-----------+-----------------+------------+


##Cross Tabs in Python
can we done wiht Pivot Tables as shown in this [Superstore Pivot](https://github.com/Praxis-QR/RDWH/blob/a1aa6c2b13fb1f315d4b19d08c4fac19064dfcdd/Pivot_Tables_for_OLAP_Cube_Python_Superstore.ipynb) Notebook

#Beyond Totals

In [ ]:
%%writefile query2.sql
SELECT
  IFNULL(Segment,'Totals') AS 'Segment, Category',       
  ttemp.Furniture, ttemp.Office_Supplies, ttemp.Technology,   
  ttemp.Furniture + ttemp.Office_Supplies + ttemp.Technology AS Totals
FROM (                                  	
  SELECT                                	
	Segment,
	AVG(IF(Category='Furniture',Sales,0)) As 'Furniture',
	AVG(IF(Category='Office Supplies',Sales,0)) As 'Office_Supplies',
	AVG(IF(Category='Technology',Sales,0)) As 'Technology'
  FROM ss_order
  GROUP BY Segment WITH ROLLUP
) AS ttemp


Writing query2.sql


In [ ]:
!mysql praxisDB --table < query2.sql

+-------------------+-----------+-----------------+------------+------------+
| Segment, Category | Furniture | Office_Supplies | Technology | Totals     |
+-------------------+-----------+-----------------+------------+------------+
| Consumer          | 75.332196 |       70.112109 |  78.289337 | 223.733642 |
| Corporate         | 75.834381 |       76.382944 |  81.606000 | 233.823325 |
| Home Office       | 68.385149 |       69.780449 | 102.806523 | 240.972121 |
| Totals            | 74.244545 |       71.947868 |  83.665609 | 229.858022 |
+-------------------+-----------+-----------------+------------+------------+


In [ ]:
%%writefile query2.sql
SELECT
  IFNULL(Segment,'Avg') AS 'Segment, Category',       
  ttemp.Furniture, ttemp.Office_Supplies, ttemp.Technology,   
  ttemp.Furniture + ttemp.Office_Supplies + ttemp.Technology AS Totals
FROM (                                  	
  SELECT                                	
	Segment,
	AVG(IF(Category='Furniture',Sales,0)) As 'Furniture',
	AVG(IF(Category='Office Supplies',Sales,0)) As 'Office_Supplies',
	AVG(IF(Category='Technology',Sales,0)) As 'Technology'
  FROM ss_order
  GROUP BY Segment WITH ROLLUP
) AS ttemp


Overwriting query2.sql


In [ ]:
!mysql praxisDB --table < query2.sql

+-------------------+-----------+-----------------+------------+------------+
| Segment, Category | Furniture | Office_Supplies | Technology | Totals     |
+-------------------+-----------+-----------------+------------+------------+
| Consumer          | 53.082951 |       69.251888 |  77.114492 | 199.449331 |
| Corporate         | 52.320874 |       70.057228 |  75.672666 | 198.050768 |
| Home Office       | 48.326259 |       71.465564 | 100.699843 | 220.491666 |
| Avg               | 52.004038 |       69.890182 |  80.886592 | 202.780812 |
+-------------------+-----------+-----------------+------------+------------+


In [ ]:
%%writefile query2.sql
SELECT
  IFNULL(Segment,'Avg') AS 'Segment, Category',       
  ttemp.Furniture, ttemp.Office_Supplies, ttemp.Technology,   
  (ttemp.Furniture + ttemp.Office_Supplies + ttemp.Technology)/3 AS Avg_xx
FROM (                                  	
  SELECT                                	
	Segment,
	AVG(IF(Category='Furniture',Sales,0)) As 'Furniture',
	AVG(IF(Category='Office Supplies',Sales,0)) As 'Office_Supplies',
	AVG(IF(Category='Technology',Sales,0)) As 'Technology'
  FROM ss_order
  GROUP BY Segment WITH ROLLUP
) AS ttemp


Overwriting query2.sql


In [ ]:
!mysql praxisDB --table < query2.sql

+-------------------+-----------+-----------------+------------+---------------+
| Segment, Category | Furniture | Office_Supplies | Technology | Avg_xx        |
+-------------------+-----------+-----------------+------------+---------------+
| Consumer          | 53.082951 |       69.251888 |  77.114492 | 66.4831103333 |
| Corporate         | 52.320874 |       70.057228 |  75.672666 | 66.0169226667 |
| Home Office       | 48.326259 |       71.465564 | 100.699843 | 73.4972220000 |
| Avg               | 52.004038 |       69.890182 |  80.886592 | 67.5936040000 |
+-------------------+-----------+-----------------+------------+---------------+


In [ ]:
%%writefile query3.sql

SELECT                                	
	Segment,
	AVG(IF(Category='Furniture',Sales,0)) As 'Furniture',
	AVG(IF(Category='Office Supplies',Sales,0)) As 'Office_Supplies',
	AVG(IF(Category='Technology',Sales,0)) As 'Technology'
  FROM ss_order
  GROUP BY Segment WITH ROLLUP

Writing query3.sql


In [ ]:
!mysql praxisDB --table < query3.sql

+-------------+-----------+-----------------+------------+
| Segment     | Furniture | Office_Supplies | Technology |
+-------------+-----------+-----------------+------------+
| Consumer    | 53.082951 |       69.251888 |  77.114492 |
| Corporate   | 52.320874 |       70.057228 |  75.672666 |
| Home Office | 48.326259 |       71.465564 | 100.699843 |
| NULL        | 52.004038 |       69.890182 |  80.886592 |
+-------------+-----------+-----------------+------------+


In [ ]:
%%writefile query3.sql
SELECT
  IFNULL(Segment,'Avg') AS 'Segment↓ Category→',       
  ttemp.Furniture, ttemp.Office_Supplies, ttemp.Technology
FROM (                                  	
  SELECT                                	
	Segment,
	AVG(IF(Category='Furniture',Sales,0)) As 'Furniture',
	AVG(IF(Category='Office Supplies',Sales,0)) As 'Office_Supplies',
	AVG(IF(Category='Technology',Sales,0)) As 'Technology'
  FROM ss_order
  GROUP BY Segment WITH ROLLUP
) AS ttemp

where Segment is not null


Overwriting query3.sql


In [ ]:
!mysql praxisDB --table < query3.sql

+------------------------+-----------+-----------------+------------+
| Segment↓ Category→     | Furniture | Office_Supplies | Technology |
+------------------------+-----------+-----------------+------------+
| Consumer               | 53.082951 |       69.251888 |  77.114492 |
| Corporate              | 52.320874 |       70.057228 |  75.672666 |
| Home Office            | 48.326259 |       71.465564 | 100.699843 |
+------------------------+-----------+-----------------+------------+


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)